# Patch

You can change the behavior of an existing function or method by patching it with `unittest.mock.patch`. You just need to specify the `target`, which refers to an object in Python. Find out more in the [specific documentation page](https://docs.python.org/3/library/unittest.mock.html#unittest.mock.patch).

In [6]:
import requests

import unittest
from unittest.mock import patch

from sklearn.neighbors import KNeighborsRegressor

## Target

There’s always some mystery associated with specifying the `target` for mocking; this section focuses on the details.

Target should be a string in the form 'package.module.ClassName'. The target is imported and the specified object replaced with the new object, so the target must be importable from the environment you are calling `patch()` from. The target is imported when the decorated function is executed, not at decoration time.

**Note**: All targets in these notebook examples begin with the `__main__` section. This allows the patched object to be referenced in the same module as the patching code.

---

The following example shows really typical case. For testing code that handles API reponse, but it's typical not ot have access to the API during code development - mocking output of the requesting tool is good option in such case.

In [10]:
def request_user(user_id):
    response = requests.get(f"https://im_not_exist/{user_id}")
    if response.ok:
        return response.text
    else:
        return "Fail!"

Patching `requests.get` to return an object with the attribute `ok=True` allows us to ensure that we receive a response in a successful case.

In [ ]:
with patch("__main__.requests.get") as mocked_get:    
    mocked_get.return_value.ok = True
    mocked_get.return_value.text = "Success"
    print(request_user("User"))

Success


Conversely, simulating an error response case returns the message corresponding to a failure.

In [12]:
with patch("__main__.requests.get") as mocked_get:
    mocked_get.return_value.ok = False
    mocked_get.return_value.text = "Success"
    print(request_user("User"))

Fail!


### Class method

You can modify the behavior of a class method as well, simply by specifying the path to it after the class name.

---

As an example, consider another common case in my practice. When writing code that needs to handle a machine learning model, you don't necessarily need to use a specific model. Mocking the model's results is a good way to verify if everything works correctly. The following cell shows how to define the result of the `predict` method of the `__main__.KNeighboursRegressor` class.

In [9]:
with patch("__main__.KNeighborsRegressor.predict") as predict:
    regressor = KNeighborsRegressor()
    predict.return_value = 'predict out'
    print(regressor.predict())

predict out


## Syntax

There are two general ways to define a patch: through the `with` context manager or the `@` decorator.

---

The following example demonstrates patching using a context manager. A function is defined, and we have details about the function call within the context manager block.

In [4]:
def my_function(a, b):
    return a + b

with patch("__main__.my_function") as p:
    my_function("hello")
    print(p.call_args)

call('hello')


### Method decoration

Decoration offers more flexibility in its usage. The function being decorated must include a special argument for the mock object, through which you can control the behavior of the target.

---

The following example demonstrates a test case where `test_method` is decorated so that any call to `hello` will be modified within it.

In [24]:
def hello(): return "hello"


class SomeTest(unittest.TestCase):
    
    @patch("__main__.hello", return_value="bye bye")
    def test_method(self, mocked_hello):
        print(hello())


ans = unittest.main(argv=[''], verbosity=0, exit=False)
del SomeTest

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


bye bye


### Class decoration

Instead of decorating just one method in a test case, you can decorate the entire test case by decorating the whole class.

---

The following cell shows applying `patch` as a decorator on the entire test case, demonstrating that all methods of the class exhibit modified behavior in this case.

In [23]:
def hello(): return "hello"

@patch("__main__.hello", return_value="bye bye")
class SomeTest(unittest.TestCase):
    def test_method1(self, mock):
        print("method1", hello())

    def test_method2(self, mock):
        print("method2", hello())

ans = unittest.main(argv=[''], verbosity=0, exit=False)

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


method1 bye bye
method2 bye bye


### Combining

The question is: if you combine different options for defining decoration, which modification will take precedence? It appears the priority, in decreasing order, is as follows: context manager, class decorator, method decorator.

---

The following cell shows a test case decorated to modify the `hello` function. However, each method within the class modifies the same function in different ways:

In [27]:
def hello(): return "hello"

@patch("__main__.hello", return_value="class decoration patching")
class SomeTest(unittest.TestCase):
    def test_method1(self, mock):
        with patch("__main__.hello", return_value="manager patching"):
            print("method1", hello())

    @patch("__main__.hello", return_value="method patching")
    def test_method2(self, mock, mock2):
        print("method2", hello())

ans = unittest.main(argv=[''], verbosity=0, exit=False)

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


method1 manager patching
method2 class decoration patching


As a result, we see messages showing the output from the context manager taking priority over the class decoration messages, and the class decoration messages taking priority over the method decoration messages.